1．$k$近邻法是基本且简单的分类与回归方法。$k$近邻法的基本做法是：对给定的训练实例点和输入实例点，首先确定输入实例点的$k$个最近邻训练实例点，然后利用这$k$个训练实例点的类的多数来预测输入实例点的类。

2．$k$近邻模型对应于基于训练数据集对特征空间的一个划分。$k$近邻法中，当训练集、距离度量、$k$值及分类决策规则确定后，其结果唯一确定。

3．$k$近邻法三要素：距离度量、$k$值的选择和分类决策规则。常用的距离度量是欧氏距离及更一般的**pL**距离。$k$值小时，$k$近邻模型更复杂；$k$值大时，$k$近邻模型更简单。$k$值的选择反映了对近似误差与估计误差之间的权衡，通常由交叉验证选择最优的$k$。

常用的分类决策规则是多数表决，对应于经验风险最小化。

4．$k$近邻法的实现需要考虑如何快速搜索k个最近邻点。**kd**树是一种便于对k维空间中的数据进行快速检索的数据结构。kd树是二叉树，表示对$k$维空间的一个划分，其每个结点对应于$k$维空间划分中的一个超矩形区域。利用**kd**树可以省去对大部分数据点的搜索， 从而减少搜索的计算量。

In [1]:
from collections import namedtuple
from operator import itemgetter
from pprint import pformat
import math
# 构造 kd树 的节点
class kdNode(namedtuple("Node","location axis leftChild rightChild")):
    def __repr__(self) -> str:
        return pformat((self.location,self.leftChild,self.rightChild))
# 递归地构造 kd树 （根据划分的轴，找中位数那个点，把空间划分为两个部分）
def kdTree(depth: int = 0, point_list:list = []):
    if not point_list:
        return None
    k = len(point_list[0])
    axis = depth % k
    point_list.sort(key = itemgetter(axis))
    median = len(point_list) // 2
    return kdNode (
        location = point_list[median],
        axis = axis,
        leftChild = kdTree(depth+1, point_list[:median]),
        rightChild = kdTree(depth+1, point_list[median+1:])
    )
    
# kd树 的前序遍历
def preOrder(root):
    print(root.location)
    if root.leftChild:
        preOrder(root.leftChild)
    if root.rightChild:
        preOrder(root.rightChild)

# 计算两点之间的距离
def distP(x, y, p=2):
    if len(x) == len(y) and len(x) > 1:
        sum = 0
        for i in range(len(x)):
            sum += math.pow((abs(x[i]-y[i])),p)
        return math.pow(sum,1/p)
    else:
        return 0

In [2]:
# 搜索kd树，寻找距离目标最近的样本 - k近邻搜索
import math
from collections import namedtuple
from random import random

result = namedtuple("result", "nearestPoint  nearestDist")

def kdSearch(kdTree, point):
    k = len(point)
    def travel(kdNode, target, maxDist):
        if kdNode is None:
            return result([0]*k, float("inf"))
        s = kdNode.axis # 当前节点划分维度
        pivot = kdNode.location # 当前节点的位置

        if target[s] <= pivot[s]: 
            nearerNode,furtherNode = kdNode.leftChild,kdNode.rightChild
        else:
            nearerNode,furtherNode = kdNode.rightChild, kdNode.leftChild

        tmp1 = travel(nearerNode, target, maxDist)

        nearest = tmp1.nearestPoint # 最近叶子节点
        dist = tmp1.nearestDist # 和最近的叶子节点距离

        maxDist = dist if dist < maxDist else maxDist  # 最近点将在以目标点为球心，max_dist为半径的超球体内
        tmpDist = abs(pivot[s] - target[s])
        
    
        if maxDist < tmpDist: # 判断超球体是否与超平面相交
            return result(nearest, dist) # 不相交则可以直接返回，不用继续判断 

        tmpDist = distP(pivot, target) 
        if tmpDist < dist:
            nearest,dist,maxDist = pivot,tmpDist,dist
        # 检查另一个子节点对应的区域是否有更近的点
        tmp2 = travel(furtherNode, target, maxDist)
        if tmp2.nearestDist < dist:
            nearest,dist = tmp2.nearestPoint,tmp2.nearestDist
        return result(nearest, dist)
    return travel(kdTree, point, float("inf"))

def main():
    point_list = [(7, 2), (5, 4), (9, 6), (4, 7), (8, 1), (2, 3)]
    target = (8.5, 5)

    tree = kdTree(0, point_list)
    print(tree)
    ret = kdSearch(tree, target)
    print(ret)

if __name__ == "__main__":
    main()

((7, 2),
 ((5, 4), ((2, 3), None, None), ((4, 7), None, None)),
 ((9, 6), ((8, 1), None, None), None))
result(nearestPoint=(9, 6), nearestDist=1.118033988749895)
